In [1]:
library(knitr)
library(sqldf)
library(ggplot2)
library(mongolite)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(tidyr)
library(jsonlite)
library(lubridate)
library(gridExtra)
library(GGally)
library(scales)
library(memisc)
library(sp)
library(rgeos)
library(rgdal)
library(maptools)
library(RColorBrewer)
library(maps)
library(grid)
library(cluster)
#library(StatMatch)
library(fastcluster)
library(factoextra)
library(stringr)
library(googleVis)
library(RMySQL)
library(data.table)
library(dbplyr)
library(DBI)
library(httr)
library(tm)
library(ldatuning)
library(udpipe)
library(egg)
library(NLP)
library(tcltk)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite

Attaching package: 'dplyr'

The following object is masked from 'package:gridExtra':

    combine

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'GGally'

The following object is masked from 'package:dplyr':

    nasa

Loading required package: lattice
Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select


Attaching package: 'memisc'

The following object is masked from 'package:scales':

    percent

The following object is masked from 'package:lubridate':

    is.interval

The following objects are masked from 'package:dplyr':

    collect, recode, rename, syms

The following objec

Further preprocessing on tweets and creation of English only POS and lemmas via udpipe

In [2]:
plot.format <- theme(axis.text = element_text(size = 14), legend.text = element_text(size = 14), legend.title = element_text(size = 14), axis.title = element_text(size = 18), plot.title = element_text(size = 18))

In [6]:
ferg_cols_loc_all <- fread(file="../Preprocessing/Files/Source/ferg_cols_loc_lexicon_all.csv", sep =",", header = TRUE, 
              stringsAsFactors=FALSE,
             showProgress=getOption("datatable.showProgress", interactive()), data.table=FALSE)

In [3]:
str(ferg_cols_loc_all)

'data.frame':	3884008 obs. of  34 variables:
 $ V1                         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ V1                         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ rowno                      : num  1 2 3 4 5 6 7 8 9 10 ...
 $ user.id                    :integer64 1539100118 9439062 63380850 1539100118 9439062 26312359 40652267 296368494 ... 
 $ user.location              : chr  "Columbus, OH" "Atlanta, GA" "CaLove" "Columbus, OH" ...
 $ user.description           : chr  "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." "Husband. Father. Prelate. Human. Company builder, 2 exits. Author @kingonomics. Inclusive ecosystem builder @th"| __truncated__ "Jealous Guardian of our Democracy #RIPTrayvonMartin #ImAlwaysWithHer" "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." ...
 $ created_at                 : chr  "Sat Aug 09 22:33:06 +0000 2014" "Sat Aug 09 22:59:13 +0000 2014" "Sat Aug 09 22:00:02 +0000 2014" "Sat Au

In [6]:
NROW(unique(ferg_cols_loc_all$full_text))

[1] 1179062

In [7]:
ferg_cols_loc_all$V1 <- NULL
ferg_cols_loc_all$V1 <- NULL

In [8]:
ferg_cols_loc_all$full_text_nohash2 <- gsub("[\\.,:;\\?!\\\\\\|\\/`=\\*\\+-^_~\\\"\\\'\\[\\]\\{\\}\\(\\)<>@$]*", " ", ferg_cols_loc_all$full_text_nohash)

In [9]:
ferg_cols_loc_all$full_text_nohash2 <- gsub("[.|:]", " ", ferg_cols_loc_all$full_text_nohash2)

In [10]:
ferg_cols_loc_all$full_text_nohash2 <- gsub("RT", "", ferg_cols_loc_all$full_text_nohash2)

In [11]:
ferg_cols_loc_all$full_text_nohash2 <- gsub("\\d{1,}", " ", ferg_cols_loc_all$full_text_nohash2)

In [12]:
ferg_cols_loc_all$full_text_nohash2 <- gsub("\\s{1,}", " ", ferg_cols_loc_all$full_text_nohash2)

In [13]:
ferg_cols_loc_all$full_text_low <- tolower(ferg_cols_loc_all$full_text_nohash2)

In [ ]:
##### Remove HTML codes

In [14]:
ferg_cols_loc_all$full_text_low <- gsub("&[a-z0-9]*;", "", ferg_cols_loc_all$full_text_low)

In [15]:
ferg_cols_loc_all$full_text_low <- gsub("[[:punct:]]", "", ferg_cols_loc_all$full_text_low)

In [ ]:
#### Remove all stopwords

In [16]:
stopwords.cust <- tm::stopwords("english")

In [ ]:
##### Remove all stop words except "not"

In [17]:
stopwords.cust <- data.frame(words = stopwords.cust)

In [18]:
stopwords.cust2 <- subset(stopwords.cust, words != "not")

In [ ]:
### Create corpus

In [19]:
corpus.low <- Corpus(VectorSource(ferg_cols_loc_all$full_text_low))

In [20]:
corpus.low <- tm_map(corpus.low, removeWords, stopwords.cust2$words)

Warning message in tm_map.SimpleCorpus(corpus.low, removeWords, stopwords.cust2$words):
"transformation drops documents"

In [22]:
ferg_cols_loc_all$full_text_low_nostop <- corpus.low$content

In [23]:
ferg_cols_loc_all$full_text_low_nostop <- trimws(ferg_cols_loc_all$full_text_low_nostop, which = "both")

In [24]:
ferg_cols_loc_all$full_text_low_nostop <- gsub("\\s{1,}", " ", ferg_cols_loc_all$full_text_low_nostop)

In [25]:
head(ferg_cols_loc_all$full_text_low_nostop)

[1] "black teenage boys not men children stop referring year old man killing children"
[2] "please tune todays killing young black male"                                     
[3] "pictures speak words unarmed teen shot times police"                             
[4] "sick tired police killing unarmed people"                                        
[5] "well go"                                                                         
[6] "damn shame coulda son condolences family"

In [ ]:
#### Check if punctuation remains

In [26]:
nrow(subset(ferg_cols_loc_all, str_detect(ferg_cols_loc_all$full_text_low, "[[:punct:]]")))

[1] 0

In [ ]:
### Placeholder to write modified text columns

write.csv(ferg_cols_loc_all, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\full_text_all.csv")

In [27]:
nrow(ferg_cols_loc_all)

[1] 3884008

In [28]:
full_text_en <- subset(ferg_cols_loc_all, lang == "en")

In [29]:
write.csv(full_text_en, "../Preprocessing/Files/Lexicon/full_text_en.csv")

In [31]:
str(full_text_en)

'data.frame':	3760412 obs. of  40 variables:
 $ rowno                      : num  1 2 3 4 5 6 7 8 9 10 ...
 $ user.id                    :integer64 1539100118 9439062 63380850 1539100118 9439062 26312359 40652267 296368494 ... 
 $ user.location              : chr  "Columbus, OH" "Atlanta, GA" "CaLove" "Columbus, OH" ...
 $ user.description           : chr  "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." "Husband. Father. Prelate. Human. Company builder, 2 exits. Author @kingonomics. Inclusive ecosystem builder @th"| __truncated__ "Jealous Guardian of our Democracy #RIPTrayvonMartin #ImAlwaysWithHer" "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." ...
 $ created_at                 : chr  "Sat Aug 09 22:33:06 +0000 2014" "Sat Aug 09 22:59:13 +0000 2014" "Sat Aug 09 22:00:02 +0000 2014" "Sat Aug 09 22:28:42 +0000 2014" ...
 $ id                         : num  4.98e+17 4.98e+17 4.98e+17 4.98e+17 4.98e+17 ...
 $ user.

In [100]:
unique(full_text_en$full_text_low_nostop)

[1] 850179

In [ ]:
### Placeholder to annotate text via udpipe - creates POS and lemmas etc. English text only

#create_udp_model <- udpipe_download_model(language = "english", 
                                       #   model_dir = "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Topics")
create_udp_model <- udpipe_load_model("../Preprocessing/Files/Lexicon/english-ud-2.0-170801.udpipe")

full_text_unq <- unique(full_text_en$full_text_low_nostop)

docs.udp.en <- udpipe_annotate(create_udp_model, x = full_text_unq)

docs.udp.en <- as.data.frame(docs.udp.en)

write.csv(docs.udp.en, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\docs_udp_en.csv")

In [ ]:
#### End of udpipe placeholder

In [ ]:
### Read in file create by udpipe

In [32]:
docs.udp.en <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\docs_udp_en.csv", stringsAsFactors = FALSE)

In [34]:
nrow(docs.udp.en)

[1] 7607581

In [35]:
docs.udp.en.lem <- docs.udp.en %>% group_by(doc_id, paragraph_id, sentence_id) %>%
                                mutate(text = str_c(lemma, collapse = " ")) %>%
                                as.data.frame()

In [37]:
str(docs.udp.en.lem)

'data.frame':	7607581 obs. of  16 variables:
 $ X            : int  1 2 3 4 5 6 7 8 9 10 ...
 $ doc_id       : chr  "doc1" "doc1" "doc1" "doc1" ...
 $ paragraph_id : int  1 1 1 1 1 1 1 1 1 1 ...
 $ sentence_id  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ sentence     : chr  "black teenage boys not men children stop referring year old man killing children" "black teenage boys not men children stop referring year old man killing children" "black teenage boys not men children stop referring year old man killing children" "black teenage boys not men children stop referring year old man killing children" ...
 $ token_id     : int  1 2 3 4 5 6 7 8 9 10 ...
 $ token        : chr  "black" "teenage" "boys" "not" ...
 $ lemma        : chr  "black" "teenage" "boy" "not" ...
 $ upos         : chr  "ADJ" "NOUN" "NOUN" "PART" ...
 $ xpos         : chr  "JJ" "NN" "NNS" "RB" ...
 $ feats        : chr  "Degree=Pos" "Number=Sing" "Number=Plur" NA ...
 $ head_token_id: int  2 5 5 5 6 7 0 7 10 11 ...
 $ dep_rel    

write.csv(docs.udp.en.lem, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\docs_udp_en_lem.csv")

docs.udp.en.lem <- read.csv("../Preprocessing/Files/Lexicon/docs_udp_en_lem.csv")

docs.udp.en.lem$X <- NULL